# Random Forest from Scratch

In this notebook, I will be entirely covering topics relating to the Random Forest Algorithm. This will start with a simple decision tree implementation, and go over various topics. The datasets that will be used are a NASA star type classfication dataset, as well as a mushroom classification dataset. 

A few things to note:

1) I am not at all concerned with performance of my algorithm in terms of speed of training. The purpose of this notebook is to ensure that I understand the decision tree and random forest algorithms well enough to implement from scratch. I don't think my implementation will ever be optimized enough to ever come close the Sklearn version of the models, but that is okay.

2) On this note, I will be using extremely small datasets for this reason. Small in this case is about 250 records or so, at a maximum. 

3) I will try to point out differences in my own implementaion vs the Sklearn when differences in performance and accuracy are seen. 

In [46]:
import pandas as pd
pd.set_option('display.max_columns', None)

## Importing and Prepping Data (Star Classification)

Temperature -- K <br>
L -- L/Lo<br>
R -- R/Ro<br>
AM -- Mv<br>
Color -- General Color of Spectrum<br>
Spectral_Class -- O,B,A,F,G,K,M / SMASS - https://en.wikipedia.org/wiki/Asteroid_spectral_types<br>
Type -- Red Dwarf, Brown Dwarf, White Dwarf, Main Sequence , Super Giants, Hyper Giants<br>

In [47]:
import numpy as np
import pandas as pd

df = pd.read_csv(r'./Stars.csv')
df.shape

(240, 7)

In [48]:
df.dtypes

Temperature         int64
L                 float64
R                 float64
A_M               float64
Color              object
Spectral_Class     object
Type                int64
dtype: object

In [49]:
df.head()

,Temperature,L,R,A_M,Color,Spectral_Class,Type
0,3068,0.002400,0.1700,16.12,Red,M,0
1,3042,0.000500,0.1542,16.60,Red,M,0
2,2600,0.000300,0.1020,18.70,Red,M,0
3,2800,0.000200,0.1600,16.65,Red,M,0
4,1939,0.000138,0.1030,20.06,Red,M,0


In [50]:
# a decision tree/ random forest can work with both categorical variables, such as color, as well
# numerical attributes, but we will need to alter our categorical attributes before passing data
# into a model. 

# for color, one of our categorical variables, we will use one hot encoding, as there is no
# inherent or ordinal relationship between the colors (one color is not greater than another). We
# will just do the same for spectral class as well.

df = pd.get_dummies(df, columns=['Color', 'Spectral_Class'], dtype=int)

# Move the "Type" column to the end
type_column = df.pop("Type")
df["Type"] = type_column

df.head()

,Temperature,L,R,A_M,Color_Blue,Color_Blue White,Color_Blue white,Color_Blue-White,Color_Blue-white,Color_Orange,Color_Orange-Red,Color_Pale yellow orange,Color_Red,Color_White,Color_White-Yellow,Color_Whitish,Color_Yellowish,Color_Yellowish White,Color_white,Color_yellow-white,Color_yellowish,Spectral_Class_A,Spectral_Class_B,Spectral_Class_F,Spectral_Class_G,Spectral_Class_K,Spectral_Class_M,Spectral_Class_O,Type
0,3068,0.002400,0.1700,16.12,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
1,3042,0.000500,0.1542,16.60,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
2,2600,0.000300,0.1020,18.70,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
3,2800,0.000200,0.1600,16.65,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
4,1939,0.000138,0.1030,20.06,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0


In [51]:
df.head()

,Temperature,L,R,A_M,Color_Blue,Color_Blue White,Color_Blue white,Color_Blue-White,Color_Blue-white,Color_Orange,Color_Orange-Red,Color_Pale yellow orange,Color_Red,Color_White,Color_White-Yellow,Color_Whitish,Color_Yellowish,Color_Yellowish White,Color_white,Color_yellow-white,Color_yellowish,Spectral_Class_A,Spectral_Class_B,Spectral_Class_F,Spectral_Class_G,Spectral_Class_K,Spectral_Class_M,Spectral_Class_O,Type
0,3068,0.002400,0.1700,16.12,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
1,3042,0.000500,0.1542,16.60,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
2,2600,0.000300,0.1020,18.70,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
3,2800,0.000200,0.1600,16.65,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
4,1939,0.000138,0.1030,20.06,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0


## Importing and Prepping Data (Mushroom Classification)

In [52]:
mush = pd.read_csv(r'./mushrooms.csv')


In [53]:
len(mush)

8124

In [54]:
mush.dtypes

class                       object
cap-shape                   object
cap-surface                 object
cap-color                   object
bruises                     object
odor                        object
gill-attachment             object
gill-spacing                object
gill-size                   object
gill-color                  object
stalk-shape                 object
stalk-root                  object
stalk-surface-above-ring    object
stalk-surface-below-ring    object
stalk-color-above-ring      object
stalk-color-below-ring      object
veil-type                   object
veil-color                  object
ring-number                 object
ring-type                   object
spore-print-color           object
population                  object
habitat                     object
dtype: object

In [55]:
import pandas as pd

# One-hot encode all columns except the first column ("class")
mush = pd.get_dummies(mush, columns=mush.columns[1:], drop_first=False)

# Display the resulting DataFrame
print(mush)


     class  cap-shape_b  cap-shape_c  cap-shape_f  cap-shape_k  cap-shape_s  \
0        p        False        False        False        False        False   
1        e        False        False        False        False        False   
2        e         True        False        False        False        False   
3        p        False        False        False        False        False   
4        e        False        False        False        False        False   
...    ...          ...          ...          ...          ...          ...   
8119     e        False        False        False         True        False   
8120     e        False        False        False        False        False   
8121     e        False        False         True        False        False   
8122     p        False        False        False         True        False   
8123     e        False        False        False        False        False   

      cap-shape_x  cap-surface_f  cap-surface_g  ca

In [56]:
from sklearn.preprocessing import LabelEncoder

# Assuming 'class' is your label column in the 'mush' DataFrame
label_encoder = LabelEncoder()

# Replace the 'class' column with numeric labels
mush['class'] = label_encoder.fit_transform(mush['class'])

In [57]:
mush.dtypes

class          int64
cap-shape_b     bool
cap-shape_c     bool
cap-shape_f     bool
cap-shape_k     bool
               ...  
habitat_l       bool
habitat_m       bool
habitat_p       bool
habitat_u       bool
habitat_w       bool
Length: 118, dtype: object

## Importing and Prepping Data (Customer Segment Classification)

In [58]:
cust = pd.read_csv(r'./Test.csv')
len(cust)


2627

In [59]:
cust = cust.drop("ID", axis=1)
cust.head()

,Gender,Ever_Married,Age,Graduated,Profession,Work_Experience,Spending_Score,Family_Size,Var_1,Segmentation
0,Female,Yes,36,Yes,Engineer,0.0,Low,1.0,Cat_6,B
1,Male,Yes,37,Yes,Healthcare,8.0,Average,4.0,Cat_6,A
2,Female,Yes,69,No,NaN,0.0,Low,1.0,Cat_6,A
3,Male,Yes,59,No,Executive,11.0,High,2.0,Cat_6,B
4,Female,No,19,No,Marketing,NaN,Low,4.0,Cat_6,A


In [60]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
cust['Segmentation'] = label_encoder.fit_transform(cust['Segmentation'])
cust.head()


,Gender,Ever_Married,Age,Graduated,Profession,Work_Experience,Spending_Score,Family_Size,Var_1,Segmentation
0,Female,Yes,36,Yes,Engineer,0.0,Low,1.0,Cat_6,1
1,Male,Yes,37,Yes,Healthcare,8.0,Average,4.0,Cat_6,0
2,Female,Yes,69,No,NaN,0.0,Low,1.0,Cat_6,0
3,Male,Yes,59,No,Executive,11.0,High,2.0,Cat_6,1
4,Female,No,19,No,Marketing,NaN,Low,4.0,Cat_6,0


In [61]:
import pandas as pd

# One-hot encode all columns except the first column ("class")
cust = pd.get_dummies(cust, columns=[col for col in cust.columns if col not in ["Age", "Work_Experience", "Family_Size", "Segmentation"]], drop_first=False)

# Display the resulting DataFrame
cust.head()

,Age,Work_Experience,Family_Size,Segmentation,Gender_Female,Gender_Male,Ever_Married_No,Ever_Married_Yes,Graduated_No,Graduated_Yes,Profession_Artist,Profession_Doctor,Profession_Engineer,Profession_Entertainment,Profession_Executive,Profession_Healthcare,Profession_Homemaker,Profession_Lawyer,Profession_Marketing,Spending_Score_Average,Spending_Score_High,Spending_Score_Low,Var_1_Cat_1,Var_1_Cat_2,Var_1_Cat_3,Var_1_Cat_4,Var_1_Cat_5,Var_1_Cat_6,Var_1_Cat_7
0,36,0.0,1.0,1,True,False,False,True,False,True,False,False,True,False,False,False,False,False,False,False,False,True,False,False,False,False,False,True,False
1,37,8.0,4.0,0,False,True,False,True,False,True,False,False,False,False,False,True,False,False,False,True,False,False,False,False,False,False,False,True,False
2,69,0.0,1.0,0,True,False,False,True,True,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,True,False
3,59,11.0,2.0,1,False,True,False,True,True,False,False,False,False,False,True,False,False,False,False,False,True,False,False,False,False,False,False,True,False
4,19,NaN,4.0,0,True,False,True,False,True,False,False,False,False,False,False,False,False,False,True,False,False,True,False,False,False,False,False,True,False


In [62]:
cust = cust.fillna(0)  # Replace NaN values with 0
cust.head()

,Age,Work_Experience,Family_Size,Segmentation,Gender_Female,Gender_Male,Ever_Married_No,Ever_Married_Yes,Graduated_No,Graduated_Yes,Profession_Artist,Profession_Doctor,Profession_Engineer,Profession_Entertainment,Profession_Executive,Profession_Healthcare,Profession_Homemaker,Profession_Lawyer,Profession_Marketing,Spending_Score_Average,Spending_Score_High,Spending_Score_Low,Var_1_Cat_1,Var_1_Cat_2,Var_1_Cat_3,Var_1_Cat_4,Var_1_Cat_5,Var_1_Cat_6,Var_1_Cat_7
0,36,0.0,1.0,1,True,False,False,True,False,True,False,False,True,False,False,False,False,False,False,False,False,True,False,False,False,False,False,True,False
1,37,8.0,4.0,0,False,True,False,True,False,True,False,False,False,False,False,True,False,False,False,True,False,False,False,False,False,False,False,True,False
2,69,0.0,1.0,0,True,False,False,True,True,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,True,False
3,59,11.0,2.0,1,False,True,False,True,True,False,False,False,False,False,True,False,False,False,False,False,True,False,False,False,False,False,False,True,False
4,19,0.0,4.0,0,True,False,True,False,True,False,False,False,False,False,False,False,False,False,True,False,False,True,False,False,False,False,False,True,False


## Decision Trees

Knowing and understanding the way a decision tree works, our decision tree from scratch will consist of the following parts:

- Node class: represents a node of the decision tree. Each node will have attributes like a split dimension (what attribute this node was split with), split point (what value of that split dimension was setteled on), and a label (which would give the class prediction if the node is a leaf), etc.

- Entropy Computation: compute the entropy of a set of labels

- Split Information: compute split information for a specific split point

- Fitting the Tree: builds the desision tree model recursively 

- Classify: uses built model to predict labels of data

### Decision Tree from Scratch

In [63]:
from typing import List, Tuple
import math
from collections import Counter
import pandas as pd


class Node:
    def __init__(self, split_dim=None, split_point=None, label=None):
        self.split_dim = split_dim
        self.split_point = split_point
        self.label = label
        self.left = None
        self.right = None
        
    def is_leaf(self):
        return self.label is not None
    
class Solution:
    def __init__(self):
        self.root = None

    # computes the information (entropy) 
    def compute_info(self, labels: pd.Series) -> float:
        total_samples = len(labels)
        label_counts = labels.value_counts()
        info = 0.0

        # this is the key line: information is the negative sum of the probablity of each class times 
        # the log of that same probablity value for each class
        for count in label_counts.values:
            probability = count / total_samples
            info -= probability * math.log2(probability)

        return info

    # grabs two subsets and tells us how valuable a split point is based on entropy
    def split_info(self, data: pd.DataFrame, label: pd.Series, split_dim: int, split_point: float) -> float:
        data_left, label_left, data_right, label_right = self._split_data(data, label, split_dim, split_point)

        total_samples = len(data)
        total_left = len(data_left)
        total_right = len(data_right)

        p_left = total_left / total_samples
        p_right = total_right / total_samples

        info_left = self.compute_info(label_left)
        info_right = self.compute_info(label_right)
        # we then weigh the information we gather from each side's split, and return the sum of those 
        # as info_a
        info_a = (p_left * info_left) + (p_right * info_right)

        return info_a

    # this is needed to actually split the data. We need to split in the recursive build, so we can define this behavior and also use it in split_info
    def _split_data(self, data: pd.DataFrame, labels: pd.Series, split_dim: int, split_point: float) -> Tuple[pd.DataFrame, pd.Series, pd.DataFrame, pd.Series]:

        # left and right just represent the two subsets that are created when we split the data. We can 
        # capture the data that is in each subset, as well as the labels
        data_left = data[data.iloc[:, split_dim] <= split_point]
        data_right = data[data.iloc[:, split_dim] > split_point]
        labels_left = labels[data.iloc[:, split_dim] <= split_point]
        labels_right = labels[data.iloc[:, split_dim] > split_point]

        return data_left, labels_left, data_right, labels_right
    
    # used to get information gain, which differs from split info
    def _calculate_info_gain(self, data: pd.DataFrame, labels: pd.Series, split_dim: int, split_point: float) -> float:
        total_samples = len(data)
        total_left, total_right = 0, 0
        count_left, count_right = {}, {}

        for i in range(total_samples):
            if data.iloc[i, split_dim] <= split_point:
                total_left += 1
                label = labels.iloc[i]
                if label in count_left:
                    count_left[label] += 1
                else:
                    count_left[label] = 1
            else:
                total_right += 1
                label = labels.iloc[i]
                if label in count_right:
                    count_right[label] += 1
                else:
                    count_right[label] = 1

        info = self.compute_info(labels)
        info_left = 0.0
        info_right = 0.0

        for count in count_left.values():
            probability = count / total_left
            info_left -= probability * math.log2(probability)

        for count in count_right.values():
            probability = count / total_right
            info_right -= probability * math.log2(probability)

        info_gain = info - ((total_left / total_samples) * info_left + (total_right / total_samples) * info_right)
        return info_gain
    
    # fit just calls recursive build
    def fit(self, train_data: pd.DataFrame, train_label: pd.Series) -> None:
        self.root = self._recursive_build(train_data, train_label)

    # builds the tree
    def _recursive_build(self, data: pd.DataFrame, labels: pd.Series, depth: int = 1) -> Node:
        label_counts = labels.value_counts()
        majority_label = label_counts.idxmax()
        
        # our stopping criteri; if the subset is all one label (leaf node) or the depth reaches more than 2, we return the node
        if len(label_counts) == 1 or depth > 4:
            return Node(label=majority_label, split_dim=-1, split_point=-1.0)

        num_features = data.shape[1]
        best_info_gain = float('-inf')
        best_split_dim = -1
        best_split_point = -1

        # find best information gain among split dimensions and split points
        for split_dim in range(num_features):
            split_points = self._calculate_split_points(data, split_dim)
            for split_point in split_points:
                info_gain = self._calculate_info_gain(data, labels, split_dim, split_point)
                if info_gain > best_info_gain:
                    best_info_gain = info_gain
                    best_split_dim = split_dim
                    best_split_point = split_point

        # set the node's properties to the split dimension, split point and label we found
        node = Node(split_dim=best_split_dim, split_point=best_split_point, label=majority_label)

        # now split again
        data_left, labels_left, data_right, labels_right = self._split_data(data, labels, best_split_dim, best_split_point)

        # if we are not at a leaf node, then call recursive build again
        if not data_left.empty and not data_right.empty:
            node.left = self._recursive_build(data_left, labels_left, depth=depth + 1)
            node.right = self._recursive_build(data_right, labels_right, depth=depth + 1)

        return node

    # needed in order to calculate the potential split points. Takes in the data and a split dimension
    def _calculate_split_points(self, data: pd.DataFrame, split_dim: int) -> List[float]:

        # gets unique values from selected dimension and sorts them in ascending order
        attribute_values = sorted(data.iloc[:, split_dim])

        # grabs all midpoints between each adjacent pair of values from our dimension
        split_points = [(attribute_values[i] + attribute_values[i + 1]) / 2 for i in range(len(attribute_values) - 1)]

        # returns the list of all split points
        return split_points
    
    # here we look into our tree we built and select the node label
    def classify(self, train_data: pd.DataFrame, train_label: pd.Series, test_data: pd.DataFrame) -> List[int]:
        self.fit(train_data, train_label)
        predictions = []

        # for each data point in the test data, set the predicted label to the value derived by traversing the tree
        for _, data_point in test_data.iterrows():
            predicted_label = self._traverse_tree(data_point, self.root)
            predictions.append(predicted_label)

        return predictions

    # look into the tree
    def _traverse_tree(self, data_point: pd.Series, node: Node) -> int:
        # if we are at leaf node, return the label
        if node.left is None and node.right is None:
            return node.label
        # if the value of the given data point (and split dimension) is less, go to the left
        if data_point[node.split_dim] <= node.split_point:
            return self._traverse_tree(data_point, node.left)
        else:
            # else, go right and call traverse again
            return self._traverse_tree(data_point, node.right)

### Testing

In [64]:
# Stars

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

features = df.drop('Type', axis=1).astype(float)  # Features
labels = df['Type'].astype(int)  # Labels

train_data, test_data, train_label, test_label = train_test_split(
    features, labels, test_size=0.2, random_state=42
)

classifier = Solution()

predictions = classifier.classify(train_data, train_label, test_data)

# print(predictions)

accuracy = accuracy_score(test_label, predictions)

print(f"Accuracy: {accuracy * 100:.2f}%")


Accuracy: 100.00%


C:\Users\reedr\AppData\Local\Temp\ipykernel_2916\1199221313.py:175: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if data_point[node.split_dim] <= node.split_point:


In [65]:
# VS SKLEARN CLASSIFIER..

from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

train_data, test_data, train_label, test_label = train_test_split(
    features, labels, test_size=0.2, random_state=42
)
dt_classifier = DecisionTreeClassifier(criterion='entropy',random_state=42)

dt_classifier.fit(train_data, train_label)

dt_predictions = dt_classifier.predict(test_data)

# print(f"Decision Tree Predictions: {dt_predictions}")

dt_accuracy = accuracy_score(test_label, dt_predictions)

print(f"Decision Tree Classifier Accuracy: {dt_accuracy * 100:.2f}%")

Decision Tree Classifier Accuracy: 100.00%


With star classification, both models perform really well. Obviously, the Sklearn implementation is orders of magnitude faster than mine, but accuracy in both cases is 100%. We can assume that due to the highly correlated varaibles with the target attribute of "Type", this data is very easy to predict with a decision tree. 

In [66]:
# Mushrooms

mush_sample = mush.sample(n=200, random_state=42)  # Set random_state for reproducibility

features = mush_sample.drop('class', axis=1).astype(int)  # Features
labels = mush_sample['class'].astype(int)  # Labels

train_data, test_data, train_label, test_label = train_test_split(
    features, labels, test_size=0.2, random_state=42
)

classifier = Solution()

predictions = classifier.classify(train_data, train_label, test_data)

# print(predictions)

accuracy = accuracy_score(test_label, predictions)

print(f"Accuracy: {accuracy * 100:.2f}%")

Accuracy: 100.00%


C:\Users\reedr\AppData\Local\Temp\ipykernel_2916\1199221313.py:175: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if data_point[node.split_dim] <= node.split_point:


In [67]:
# VS SKLEARN CLASSIFIER..

features = mush.drop('class', axis=1).astype(int)  # Features
labels = mush['class'].astype(int)  # Labels

train_data, test_data, train_label, test_label = train_test_split(
    features, labels, test_size=0.2, random_state=42
)

dt_classifier = DecisionTreeClassifier(criterion='entropy',random_state=42)

dt_classifier.fit(train_data, train_label)

dt_predictions = dt_classifier.predict(test_data)

# print(f"Decision Tree Predictions: {dt_predictions}")

dt_accuracy = accuracy_score(test_label, dt_predictions)

print(f"Decision Tree Classifier Accuracy: {dt_accuracy * 100:.2f}%")

Decision Tree Classifier Accuracy: 100.00%


I think I need to test this on a dataset that is less correlative with the target attribute. Maybe Something harder to predict?

In [75]:
# Customer Data

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

cust_sample = cust.sample(n=500, random_state=42)  # Set random_state for reproducibility

features = cust_sample.drop('Segmentation', axis=1).astype(int)  # Features
labels = cust_sample['Segmentation'].astype(int)  # Labels

train_data, test_data, train_label, test_label = train_test_split(
    features, labels, test_size=0.2, random_state=42
)

classifier = Solution()

predictions = classifier.classify(train_data, train_label, test_data)

# print(predictions)

accuracy = accuracy_score(test_label, predictions)

print(f"Accuracy: {accuracy * 100:.2f}%")

Accuracy: 34.00%


C:\Users\reedr\AppData\Local\Temp\ipykernel_2916\1199221313.py:175: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if data_point[node.split_dim] <= node.split_point:


In [76]:
# VS SKLEARN CLASSIFIER..

features = cust_sample.drop('Segmentation', axis=1).astype(int)  # Features
labels = cust_sample['Segmentation'].astype(int)  # Labels

train_data, test_data, train_label, test_label = train_test_split(
    features, labels, test_size=0.2, random_state=42
)

dt_classifier = DecisionTreeClassifier(criterion='entropy',random_state=42)

dt_classifier.fit(train_data, train_label)

dt_predictions = dt_classifier.predict(test_data)

# print(f"Decision Tree Predictions: {dt_predictions}")

dt_accuracy = accuracy_score(test_label, dt_predictions)

print(f"Decision Tree Classifier Accuracy: {dt_accuracy * 100:.2f}%")

Decision Tree Classifier Accuracy: 28.00%


So I actually perform better than the sklearn model with small datasets..

### 

## Random Forests

### Random Forest as an Ensemble of Decision Trees

### Random Subspace Method for Feature Selection

### Bootstrapping 

### Hyperparameter Tuning

## Variations and Improvements

### Feature Importance Estimation with Random Forest

### Extending Random Forest to Handle Imbalanced Datasets

### Multiclass Classification and Regression Problems